In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import os
#PROJECT_ROOT = '/content/drive/My Drive/tfm/codigo' #@param {type:"string"}
#os.chdir(PROJECT_ROOT)
#print("Current working directory:", os.getcwd())

Current working directory: /content/drive/My Drive/tfm/codigo


In [ ]:
import numpy as np
import pandas as pd

import joblib

In [ ]:
# === Cargar modelo y escaladores ===
model = joblib.load("/content/drive/My Drive/tfm/codigo/modelos_exportados/modelo_ml/modelo_xgb.pkl")

scaler_entrada = joblib.load("/content/drive/My Drive/tfm/codigo/modelos_exportados/standardscaler_datos_entrada.pkl")
scaler_precio = joblib.load("/content/drive/My Drive/tfm/codigo/modelos_exportados/standardscaler_precio.pkl")


In [6]:
# === Input manual ===
# El precio_medio_mun_tipo ya debe estar en euros → le aplicas log
# Todas las variables deben coincidir con las columnas usadas para entrenar
# Aqui la Capital no se usa

input_dict = {
    'Habitaciones': 3,
    'Aseos': 2,
    'Metros': 105,
    'CUDIS': 2408907,
    'Población': 120000,
    'Renta bruta media por persona': 36000,
    'Comodidades': 1,
    'Capital': 1,
    'Precio_medio_mun_tipo': np.log(132000)  # € → aplicar log
}


In [29]:
# === Preprocesado ===

# 1. Define the column order (including 'Capital')
column_order = ['Habitaciones', 'Aseos', 'Metros', 'CUDIS', 'Población',
                'Renta bruta media por persona', 'Comodidades', 'Capital',
                'Precio_medio_mun_tipo']

# 2. Features to be scaled (excluding 'Capital')
features_to_scale = [feat for feat in column_order if feat != 'Capital']

# 3. Create input data for scaling
X_input_to_scale = pd.DataFrame([input_dict], columns=features_to_scale)

# 4. Scale the selected features
X_scaled = scaler_entrada.transform(X_input_to_scale)

# 5. Insert 'Capital' at the penultimate position
capital_index = column_order.index('Capital')
X_with_capital = np.insert(X_scaled, capital_index, input_dict['Capital'], axis=1)
print(X_with_capital)

[[ 0.24348198  0.87927311  0.36340549 -0.44528777  0.51035166  1.05159444
   0.90479803  1.          0.54688789]]


In [22]:
# === Predicción ===
y_pred_scaled = model.predict(X_scaled)

# === Inversión de escalado + exponencial ===
# Reshape y_pred_scaled to a 2D array with a single column
y_pred_log = scaler_precio.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_pred_eur = np.exp(y_pred_log)

precio_estimado = y_pred_eur[0][0]
print(f"📦 Precio estimado: {precio_estimado:,.2f} €")
print(f"El rango de precios 💰 este inmueble es: {(precio_estimado*0.7):,.2f} € y {(precio_estimado*1.3):,.2f} €")

📦 Precio estimado: 137,782.45 €
El rango de precios 💰 este inmueble es: 96,447.72 € y 179,117.19 €


In [30]:
import numpy as np
import pandas as pd

def predecir_precio(input_dict, scaler_entrada, scaler_precio, model):
    """
    Predice el precio final en euros a partir de un input_dict
    con características de la vivienda, usando un modelo XGBoost.

    Parámetros:
        input_dict: Diccionario con los valores de entrada (en euros y sin escalar).
        scaler_entrada: StandardScaler entrenado para X (entrada).
        scaler_precio: StandardScaler entrenado para y (salida en log).
        model: Modelo XGBoost entrenado.

    Retorna:
        Precio estimado (en euros).
    """

    # Orden de las columnas (incluyendo 'Capital')
    column_order = ['Habitaciones', 'Aseos', 'Metros', 'CUDIS', 'Población',
                    'Renta bruta media por persona', 'Comodidades',
                    'Precio_medio_mun_tipo']

    # Asegurar que el precio medio ya está en log en el input_dict
    input_dict['Precio_medio_mun_tipo'] = np.log(input_dict['Precio_medio_mun_tipo'])

    # Crear DataFrame con el orden correcto
    X_input_df = pd.DataFrame([input_dict], columns=column_order)

    # Features a escalar (excluyendo 'Capital')
    features_to_scale = [feat for feat in column_order if feat != 'Capital']

    # Crear datos de entrada para escalar
    X_input_to_scale = pd.DataFrame([input_dict], columns=features_to_scale)

    # Escalar las características seleccionadas
    X_scaled = scaler_entrada.transform(X_input_to_scale)

    # Predecir con el modelo XGBoost
    y_pred_scaled = model.predict(X_scaled)

    # Invertir escalado + exponencial
    y_pred_log = scaler_precio.inverse_transform(y_pred_scaled.reshape(-1, 1))
    y_pred_eur = np.exp(y_pred_log)

    precio_estimado = y_pred_eur[0][0]
    return precio_estimado

In [31]:
input_dict = {
    'Habitaciones': 3,
    'Aseos': 2,
    'Metros': 105,
    'CUDIS': 2408907,
    'Población': 120000,
    'Renta bruta media por persona': 36000,
    'Comodidades': 1,
    'Capital': 1,
    'Precio_medio_mun_tipo': 132000  # en euros → la función aplica log
}

precio_estimado = predecir_precio(input_dict, scaler_entrada, scaler_precio, model)
print(f"💰 Precio estimado: {precio_estimado:,.2f} €")
print(f"El rango de precios 💰 este inmueble es: {(precio_estimado*0.7):,.2f} € y {(precio_estimado*1.3):,.2f} €")

💰 Precio estimado: 137,782.45 €
El rango de precios 💰 este inmueble es: 96,447.72 € y 179,117.19 €
